### Module for teaching dataset generation
This module contains the important parameters needed in the generation of teaching dataset. The important part of this module is the class `pole` which creates an object that contains the pole position, Riemann sheet location, provides sshadow pole information and a method `pole.smat` that gives the pole's S-matrix contribution.


In [19]:
#4*2120*2120 #(17977600) #35*720*720 #(18144000)

17977600

In [10]:
import math
import numpy as np
import cmath as cm
import pandas as pd
import random
import pickle
import os

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import chainer
from chainer import configuration
from chainer.dataset import convert
import sklearn
from sklearn.utils import shuffle
from tabulate import tabulate

We now define the global parameters used in generating the dataset. This includes the hadron masses and the center-of-mass energy $E_{cm}$ where the cross-section is to be plotted.

In [ ]:
####################################
#Units in MeV
Nucleon = 938.9186795
Pion = 138.0394
Eta  = 547.862 

mu_NN   = 1/(1/Nucleon + 1/Nucleon)
mu_piN  = 1/(1/Pion + 1/Nucleon)
mu_etaN = 1/(1/Eta + 1/Nucleon)

T_piN  = Nucleon + Pion
T_etaN = Nucleon + Eta
T_KLam = 1611.42
T_KSig = 1688.32

#choose system by specifying reduced mass (convert to fm^-1)
mu1 = mu_piN/(197.3)
mu2 = mu_etaN/(197.3)

T1 = T_piN/(197.3)
T2 = T_etaN/(197.3)
T3 = T_KLam/(197.3)
T4 = T_KSig/(197.3)

#Number of real and imaginary parts of the pole
#Choose even numbers only
Nreal = 2120
Nimag = 2120

#Generate Npole poles within the counting region
#units in MeV
Erealbelow = np.random.uniform(low=T_etaN-50, high=T_etaN, size=int(Nreal/2))
Erealabove = np.random.uniform(low=T_etaN, high=T_KSig, size=int(Nreal/2))
Ereal = np.concatenate((Erealbelow, Erealabove))
Eimag = np.random.uniform(low=0.0, high=200.0, size=Nimag)
    
#Generate poles beyond the counting region
#units in MeV
Erealbelow = np.random.uniform(low=T_piN-2000, high=T_piN-100, size=int(Nreal/2))
Erealabove = np.random.uniform(low=T_etaN+500, high=T_KSig+5000, size=int(Nreal/2))
Erealfar = np.concatenate((Erealbelow, Erealabove))
Eimagfar = np.random.uniform(low=700.0, high=2000.0, size=Nimag)  


In [ ]:
#inspect = True
inspect = False
directory = 'curriculum01_training'
#curr01 datasets: 00, 01, 11, 21

#directory = 'curriculum02_training'
#curr02 datasets: 00, 01, 11, 21, 02
#directory = 'curriculum03_training'
#curr03 datasets: 00, 01, 11, 21, 02, 12
#directory = 'curriculum04_training'
#curr04 datasets: 00, 01, 11, 21, 02, 12, 22
#directory = 'curriculum05_training'
#curr05 datasets: 00, 01, 11, 21, 02, 12, 22, 10
#directory = 'curriculum06_training'
#curr06 datasets: 00, 01, 11, 21, 02, 12, 22, 10, 20
#directory = 'curriculum07_training'
#curr07 datasets: 00, 01, 11, 21, 02, 12, 22, 10, 20, 30


#directory = 'curriculum08_training'
#curr08 datasets: 00, 01, 11, 21, 02, 12, 22, 10, 20, 30, 03

#directory = 'curriculum32_training'
#all datasets


#directory = 'sample_plot'

if not os.path.isdir(directory):
    os.makedirs(directory)
print('Number of poles to be generated per class:', Nreal*Nimag)
print('Ndata to be generated=', 4*Nreal*Nimag)
print('Your directory is:', directory)

Below is the output label for DNN RS pole-counter (maximum of 4 poles only)

In [ ]:
#descriptive labels of network output
#at most 4 poles in all RS
labelz = [
#default no pole
    'no nearby pole',                          #00
#poles in [bt]    
    '1 pole  in [bt]',                          #01
    '2 poles in [bt]',                         #02
    '3 poles in [bt]',                         #03
    '4 poles in [bt]',                         #04
#[bt] and [bb] no shadow pair    
    '3 poles in [bt] and 1 pole  in [bb]',      #05
    '2 poles in [bt] and 1 pole  in [bb]',      #06
    '2 poles in [bt] and 2 poles in [bb]',     #07 
    '1 pole  in [bt] and 2 poles in [bb]',      #08
    '1 pole  in [bt] and 3 poles in [bb]',     #09
    '1 pole  in [bt] and 1 pole  in [bb]',      #10
#poles in [bb] only    
    '1 pole  in [bb]',                          #11
    '2 poles in [bb]',                         #12
    '3 poles in [bb]',                         #13
    '4 poles in [bb]',                         #14
#[bb] and [tb] no shadow pair    
    '3 poles in [bb] and 1 pole  in [tb]',      #15
    '2 poles in [bb] and 1 pole  in [tb]',      #16
    '2 poles in [bb] and 2 poles in [tb]',     #17   
    '1 pole  in [bb] and 2 poles in [tb]',      #18
    '1 pole  in [bb] and 3 poles in [tb]',     #19
    '1 pole  in [bb] and 1 pole  in [tb]',      #20
#poles in [tb] only    
    '1 pole  in [tb]',                          #21
    '2 poles in [tb]',                         #22
    '3 poles in [tb]',                         #23
    '4 poles in [tb]',                         #24    
#[tb] and [bt]
    '3 poles in [tb] and 1 pole  in [bt]',      #25
    '2 poles in [tb] and 1 pole  in [bt]',      #26
    '2 poles in [tb] and 2 poles in [bt]',     #27
    '1 pole  in [tb] and 2 poles in [bt]',      #28
    '1 pole  in [tb] and 3 poles in [bt]',     #29 
    '1 pole  in [tb] and 1 pole  in [bt]',      #30      
#poles in all three
    '2 poles in [bt] and 1 pole  in [bb] and 1 pole  in [tb]',    #31
    '1 pole  in [bt] and 2 poles in [bb] and 1 pole  in [tb]',    #32
    '1 pole  in [bt] and 1 pole  in [bb] and 2 poles in [tb]',    #33
    '1 pole  in [bt] and 1 pole  in [bb] and 1 pole  in [tb]'      #34
]

We use the energy spacing used in the known PWA of pion-nucleon system such as the <a href=http://gwdac.phys.gwu.edu/analysis/pin_analysis.html>SAID-GWU</a> model, the 
<a href=http://collaborations.fz-juelich.de/ikp/meson-baryon/juelich_amplitudes.html>Julich-Bonn </a> model and the <a href=https://www.phy.anl.gov/theory/research/anl-osaka-pwa/>ANL-Osaka</a> model. 
It is preferrable to use the <a href=http://gwdac.phys.gwu.edu/analysis/pin_analysis.html>SAID-GWU</a> mode since it also has uncertainties in the single-energy value solutions. We load the data below:

In [ ]:
#SAID-GWU model
#Center-of-mass energy axis (in MeV) and the real and imaginary parts of scattering amplitude
#Data points not beyond K-Lambda threshold
piN_data = pd.read_excel("S11-piN.xlsx", 
                         sheet_name="S11-SAID", 
                         usecols=[2,7,9], 
                         skiprows=[0],
                         nrows=37)  #30-upto K-Lambda only
E_exp = piN_data["Ecm"].tolist()
Treal = piN_data["Trl"].tolist()
Timag = piN_data["Tim"].tolist()

In [ ]:
#Note that the energy points of the moch data are randomly selected in the desired energy range.
#We only use the experimental data to match the number of energy bins.
E_exp = np.array(E_exp)
Treal = np.array(Treal)
Timag = np.array(Timag)
Tsqr = np.square(Treal) + np.square(Timag)

In [ ]:
#We will be using several assigned poles in the calculation of S-matrix
#To simplify the code we introduced the class "pole" where the user can 
#assign the Riemann sheet location and the pole's real and imaginary parts.
#The class pole will throw away the resulting shadow pole by choosing 
#the appropriate ZCL model. Information about the shadow pole can be accessed
#by the attribute xxx.shadow
#We define a method xxx.smat() to the class pole that gives its contribution to the
#overall S-matrix

class pole:
    def __init__(self, RS, Ereal, Eimag):
        #RS is the two-channel Reimann sheet location of the pole
        #Use [1,-1] for bt, [-1,-1] for bb and [-1,1] for tb
        #Ereal is the real part of energy pole
        #Eimag is the imag part of energy pole
        Epole = Ereal - (1j)*Eimag
        self.pos = Epole
        #compute momentum pole
        k1pole = cm.sqrt(2.0*mu1*(Epole/197.3-T1))
        k2pole = cm.sqrt(2.0*mu2*(Epole/197.3-T2))
        alpha1 = np.abs(k1pole.real)
        alpha2 = np.abs(k2pole.real)
        beta1  = RS[0]*abs(k1pole.imag)
        beta2  = RS[1]*abs(k2pole.imag)
        
        self.alpha1 = alpha1
        self.alpha2 = alpha2
        self.beta1 = beta1
        self.beta2 = beta2
        
        #coupling for model1: resonance initially coupled to channel 1
        lambdaplus_model1 = (-(2.0*mu1*mu2*alpha1**2.0+4.0*mu1**2*beta2**2.0) \
                             +np.sqrt((2.0*mu1*mu2*alpha1**2+4.0*mu1**2.0*beta2**2.0)**2.0 \
                                      -4.0*(mu1*mu2*alpha1**2.0)**2.0))/(2.0*(alpha1*mu2)**2.0)
        lambdaminus_model1 = (-(2.0*mu1*mu2*alpha1**2.0+4.0*mu1**2*beta2**2) \
                              -np.sqrt((2*mu1*mu2*alpha1**2+4*mu1**2*beta2**2)**2.0 \
                                       -4.0*(mu1*mu2*alpha1**2.0)**2.0))/(2*(alpha1*mu2)**2.0)
        #coupling for model2: resonance initially coupled to channel 2
        lambdaplus_model2 = (-(2.0*mu1*mu2*alpha2**2.0+4.0*mu2**2.0*beta1**2.0) \
                             +np.sqrt((2.0*mu1*mu2*alpha2**2.0+4.0*mu2**2.0*beta1**2.0)**2.0 \
                                      -4.0*(mu1*mu2*alpha2**2.0)**2.0))/(2.0*(alpha2*mu1)**2.0)
        lambdaminus_model2 = (-(2.0*mu1*mu2*alpha2**2.0+4.0*mu2**2.0*beta1**2.0) \
                              -np.sqrt((2.0*mu1*mu2*alpha2**2.0+4.0*mu2**2.0*beta1**2.0)**2.0 \
                                       -4.0*(mu1*mu2*alpha2**2.0)**2.0))/(2*(alpha2*mu1)**2.0)        
        
        #the choice of model depends on the assigned Riemann sheet
        if RS==[-1,1]:  #[bt] or second Riemann sheet
            #use initially-coupled to channel 1 model
            model = 1
            lambdaplus = lambdaplus_model1
            lambdaminus = lambdaminus_model1
        elif RS==[1,-1] or RS==[-1,-1]:  #[tb] or fourth Riemann sheet/ [bb] or third Riemann sheet
            #use initially-coupled to channel 2 model
            model = 2
            lambdaplus = lambdaplus_model2
            lambdaminus = lambdaminus_model2
        #Note on [bb]: It is safe to use model 2 for [bb] since the shadow is guaranteed to be in [tb],
        #and is always below the assigned pole. This makes the shadow irrelevant in the second threshold region
             
        
        #Check if the pole is relevant
        if Eimag>500:
            #if pole is background, use weak coupling
            #shadow pole here will also be irrelevant
            lambdacoup = np.random.uniform(low=0.5*lambdaplus, high=-0.5*lambdaplus) 
        elif Eimag<500:
            #if poles is relevant, use strong coupling to
            #throw shadow poles
            lambdacoup = np.random.uniform(low=0.5*(lambdaminus-lambdaplus)+lambdaplus, high=0.99*lambdaplus)
            if Eimag<10:
                lambdacoup = np.random.uniform(low=lambdaminus, high=lambdaplus)
        #the official coupling:
        self.lambdacoup = lambdacoup
        #Let us calculate the shadow pole
        if model == 1:
            shadowp1 = (1j)*beta1*((mu1-mu2*lambdacoup)/(mu1+mu2*lambdacoup))+cm.sqrt(alpha1**2.0+4.0*lambdacoup*mu1**2.0*beta2**2.0/(mu1+mu2*lambdacoup)**2.0)
            shadowp2 = (1j)*beta2*(-(mu1-mu2*lambdacoup)/(mu1+mu2*lambdacoup))+cm.sqrt(alpha2**2.0+4.0*lambdacoup*mu2**2.0*beta1**2.0/(mu1+mu2*lambdacoup)**2.0)
            shadowm1 = (1j)*beta1*((mu1-mu2*lambdacoup)/(mu1+mu2*lambdacoup))-cm.sqrt(alpha1**2.0+4.0*lambdacoup*mu1**2.0*beta2**2.0/(mu1+mu2*lambdacoup)**2.0)
            shadowm2 = (1j)*beta2*(-(mu1-mu2*lambdacoup)/(mu1+mu2*lambdacoup))-cm.sqrt(alpha2**2.0+4.0*lambdacoup*mu2**2.0*beta1**2.0/(mu1+mu2*lambdacoup)**2.0)
        elif model == 2:
            shadowp1 = (1j)*beta1*((mu1*lambdacoup-mu2)/(mu1*lambdacoup+mu2))+cm.sqrt(alpha1**2.0+4.0*lambdacoup*mu1**2.0*beta2**2.0/(mu1*lambdacoup+mu2)**2.0)
            shadowp2 = (1j)*beta2*(-(mu1*lambdacoup-mu2)/(mu1*lambdacoup+mu2))+cm.sqrt(alpha2**2.0+4.0*lambdacoup*mu2**2.0*beta1**2.0/(mu1*lambdacoup+mu2)**2.0)
            shadowm1 = (1j)*beta1*((mu1*lambdacoup-mu2)/(mu1*lambdacoup+mu2))-cm.sqrt(alpha1**2.0+4.0*lambdacoup*mu1**2.0*beta2**2.0/(mu1*lambdacoup+mu2)**2.0)
            shadowm2 = (1j)*beta2*(-(mu1*lambdacoup-mu2)/(mu1*lambdacoup+mu2))-cm.sqrt(alpha2**2.0+4.0*lambdacoup*mu2**2.0*beta1**2.0/(mu1*lambdacoup+mu2)**2.0)
        
        #Riemann sheet identifier for shadow pole
        def RSlabel(pimag1, pimag2):
            if pimag1>0 and pimag2>0:
                RS = 'tt'
            elif pimag1<0 and pimag2>0:
                RS = 'bt'
            elif pimag1<0 and pimag2<0:
                RS = 'bb'
            elif pimag1>0 and pimag2<0:
                RS = 'tb'
            return RS
        #Attributes for the shadow pole (for inspection purposes only)
        #This will give the energy shadow pole in MeV with its RS location
        self.shadow = ['{:.2f}'.format((shadowp1**2.0/(2.0*mu1)+T1)*197.3), RSlabel(shadowp1.imag, shadowp2.imag),  
                       '{:.2f}'.format((shadowm1**2.0/(2.0*mu1)+T1)*197.3), RSlabel(shadowm1.imag, shadowm2.imag)]
        #Riemann sheet of the pole
        self.RS = RSlabel(beta1, beta2)
        #The model used for this pole (needed in the method part)
        self.model = model
    def smat(self,p1,p2):
        #this is the method to compute S-matrix contribution of the pole
        #all parameters must be defined as attribute (self)
        if self.model == 1:
            Dnum = ((-p1-(1j)*self.beta1)**2.0 - self.alpha1**2.0)+self.lambdacoup*((p2-(1j)*self.beta2)**2.0 - self.alpha2**2.0)
            Dden = ((p1-(1j)*self.beta1)**2.0 - self.alpha1**2.0)+self.lambdacoup*((p2-(1j)*self.beta2)**2.0 - self.alpha2**2.0)
        elif self.model == 2:
            Dnum = self.lambdacoup*((-p1-(1j)*self.beta1)**2.0 - self.alpha1**2.0)+((p2-(1j)*self.beta2)**2.0 - self.alpha2**2.0)
            Dden = self.lambdacoup*((p1-(1j)*self.beta1)**2.0 - self.alpha1**2.0)+((p2-(1j)*self.beta2)**2.0 - self.alpha2**2.0)
        smatcontribution = Dnum/Dden    
        return (smatcontribution)        

In [ ]:
def skipduplicate(real1, imag1, Nreal, Nimag):
    #This is to avoid possible duplication of poles in the same Riemann sheet
    real_list = list(range(1,Nreal))
    imag_list = list(range(1,Nimag))
    real2 = np.random.choice([entry for entry in real_list if entry != real1]) 
    real3 = np.random.choice([entry for entry in real_list if entry != real1 and entry != real2])
    real4 = np.random.choice([entry for entry in real_list if entry != real1 and entry != real2 and entry != real3])
    real5 = np.random.choice([entry for entry in real_list if entry != real1 and entry != real2 and entry != real3 and entry != real4])
    real6 = np.random.choice([entry for entry in real_list if entry != real1 and entry != real2 and entry != real3 and entry != real4 and entry != real5])
    imag2 = np.random.choice([entry for entry in imag_list if entry != imag1])
    imag3 = np.random.choice([entry for entry in imag_list if entry != imag1 and entry != imag2])
    imag4 = np.random.choice([entry for entry in imag_list if entry != imag1 and entry != imag2 and entry != imag3])
    imag5 = np.random.choice([entry for entry in imag_list if entry != imag1 and entry != imag2 and entry != imag3 and entry != imag4])
    imag6 = np.random.choice([entry for entry in imag_list if entry != imag1 and entry != imag2 and entry != imag3 and entry != imag4 and entry != imag5])
    indx = [[real1, real2, real3, real4, real5, real6],[imag1, imag2, imag3, imag4, imag5, imag6]]
    return indx 

In [ ]:
def seerealimagpart(Einput, ReT11, ImT11,labelout, data_info):
    #chckind is a random integer to call one of the generated data sample
    chckind = np.random.randint(0,len(labelout))
    font_set_size = 15
    Thres1, ValatThres1 = [T_piN, T_piN], [-max(max(ReT11[chckind]**2.0+ImT11[chckind]**2.0),max(np.abs(ReT11[chckind])),max(np.abs(ImT11[chckind]))), max(max(ReT11[chckind]**2.0+ImT11[chckind]**2.0),max(np.abs(ReT11[chckind])),max(np.abs(ImT11[chckind])))]
    Thres2, ValatThres2 = [T_etaN, T_etaN], [-max(max(ReT11[chckind]**2.0+ImT11[chckind]**2.0),max(np.abs(ReT11[chckind])),max(np.abs(ImT11[chckind]))), max(max(ReT11[chckind]**2.0+ImT11[chckind]**2.0),max(np.abs(ReT11[chckind])),max(np.abs(ImT11[chckind])))]
    Thres3, ValatThres3 = [T_KLam, T_KLam], [-max(max(ReT11[chckind]**2.0+ImT11[chckind]**2.0),max(np.abs(ReT11[chckind])),max(np.abs(ImT11[chckind]))), max(max(ReT11[chckind]**2.0+ImT11[chckind]**2.0),max(np.abs(ReT11[chckind])),max(np.abs(ImT11[chckind])))]
    Thres4, ValatThres4 = [T_KSig, T_KSig], [-max(max(ReT11[chckind]**2.0+ImT11[chckind]**2.0),max(np.abs(ReT11[chckind])),max(np.abs(ImT11[chckind]))), max(max(ReT11[chckind]**2.0+ImT11[chckind]**2.0),max(np.abs(ReT11[chckind])),max(np.abs(ImT11[chckind])))]
    Horx, Hory = [T_piN, T_KSig], [0, 0]
    plt.ylim(-max(max(ReT11[chckind]**2.0+ImT11[chckind]**2.0),max(np.abs(ReT11[chckind])),max(np.abs(ImT11[chckind])))-0.05,max(max(ReT11[chckind]**2.0+ImT11[chckind]**2.0),max(np.abs(ReT11[chckind])),max(np.abs(ImT11[chckind])))+0.05)
    #plt.ylim(-1.05,1.05)
    plt.plot(Horx, Hory,'red')
    plt.plot(Thres1, ValatThres1,'red')
    plt.plot(Thres2, ValatThres2,'red')
    plt.plot(Thres3, ValatThres3,'red')
    plt.plot(Thres4, ValatThres4,'red')
    
    plt.plot(Einput[chckind], ReT11[chckind],'+')
    plt.plot(Einput[chckind], ImT11[chckind],'*')
    plt.plot(Einput[chckind], ReT11[chckind]**2.0+ImT11[chckind]**2.0,'o')
    plt.title('input data', fontsize=font_set_size)
    plt.xlabel('$E_{cm}$ (MeV)', fontsize=font_set_size)
    plt.xticks(fontsize=font_set_size)
    plt.ylabel('$Re T_{11}$, $Im T_{11}$', fontsize=font_set_size)
    plt.yticks(fontsize=font_set_size)
    plt.tight_layout()
    isactive = ['No', 'Yes']
    
    print('class','{:02d}'.format(labelout[chckind]),':',labelz[labelout[chckind]])
    teburu = [
    ['n','energy pole (MeV)', 'RS', 'Active','shadow1 (MeV)', 'RS1', 'shadow2 (MeV)', 'RS2'],
    ['1','{:.2f}'.format(data_info[chckind][0][0]), data_info[chckind][1][0], isactive[data_info[chckind][3][0]], data_info[chckind][2][0][0], data_info[chckind][2][0][1], data_info[chckind][2][0][2], data_info[chckind][2][0][3]],
    ['2','{:.2f}'.format(data_info[chckind][0][1]), data_info[chckind][1][1], isactive[data_info[chckind][3][1]], data_info[chckind][2][1][0], data_info[chckind][2][1][1], data_info[chckind][2][1][2], data_info[chckind][2][1][3]],
    ['3','{:.2f}'.format(data_info[chckind][0][2]), data_info[chckind][1][2], isactive[data_info[chckind][3][2]], data_info[chckind][2][2][0], data_info[chckind][2][2][1], data_info[chckind][2][2][2], data_info[chckind][2][2][3]],
    ['4','{:.2f}'.format(data_info[chckind][0][3]), data_info[chckind][1][3], isactive[data_info[chckind][3][3]], data_info[chckind][2][3][0], data_info[chckind][2][3][1], data_info[chckind][2][3][2], data_info[chckind][2][3][3]], 
    ['5','{:.2f}'.format(data_info[chckind][0][4]), data_info[chckind][1][4], isactive[data_info[chckind][3][4]], data_info[chckind][2][4][0], data_info[chckind][2][4][1], data_info[chckind][2][4][2], data_info[chckind][2][4][3]],
    ['6','{:.2f}'.format(data_info[chckind][0][5]), data_info[chckind][1][5], isactive[data_info[chckind][3][5]], data_info[chckind][2][5][0], data_info[chckind][2][5][1], data_info[chckind][2][5][2], data_info[chckind][2][5][3]]
    ]
    
    print(tabulate(teburu))
    return plt.show(), chckind

In [ ]:
def exportdata(Einput, ReT11, ImT11, labelout, data_info, directoryint):
    #specify directory of files to be exported
    #use test_Xpoles or train_Xpoles in directory name
    out = directoryint
    if not os.path.isdir(out):
        os.makedirs(out)
    #EXPORT generated data
    pickle.dump (Einput, open(os.path.join(out,'Einput.pkl'),'wb'), protocol=4)
    pickle.dump (ReT11, open(os.path.join(out,'ReT11.pkl'),'wb'), protocol=4)
    pickle.dump (ImT11, open(os.path.join(out,'ImT11.pkl'),'wb'), protocol=4)
    pickle.dump (labelout, open(os.path.join(out,'labelout.pkl'),'wb'), protocol=4)
    pickle.dump (data_info, open(os.path.join(out,'data_info.pkl'),'wb'), protocol=4)
    
    #The following will collect the data that will go to the input layer:
    #You can design a DNN with Einput and T11sqr in the input layer
    T11sqr = [x**2.0 + y**2.0 for x, y in zip(ReT11, ImT11)]
    T11sqr = np.concatenate((Einput,T11sqr), axis=1)
    #or you can design a DNN with Einput, ReT11 and ImT11 in the input layer
    T11 = np.concatenate((Einput,ReT11, ImT11), axis=1)
    pickle.dump (T11sqr, open(os.path.join(out,'T11sqr.pkl'),'wb'), protocol=4)
    pickle.dump (T11, open(os.path.join(out,'T11.pkl'),'wb'), protocol=4)
    print('export done')        
    return

In [ ]:
def importdata(directoryint):
    #import dataset for inspection
    out = directoryint
    Einput = pickle.load(open(os.path.join(out,'Einput.pkl'),'rb'))
    ReT11 = pickle.load(open(os.path.join(out,'ReT11.pkl'),'rb'))
    ImT11 = pickle.load(open(os.path.join(out,'ImT11.pkl'),'rb'))
    labelout = pickle.load(open(os.path.join(out,'labelout.pkl'),'rb'))
    data_info= pickle.load(open(os.path.join(out,'data_info.pkl'),'rb'))
    return Einput, ReT11, ImT11, labelout, data_info

In [ ]:
def get_traintest(Nshuffle, directory):
    #If Nshuffle=0, it is understood that the testing dataset is to be prepared.
    #Otherwise, it is the training dataset and Nshuffle determines shuffling times.
    out = directory
    #import prepared dataset
    #inputtraining = pickle.load(open('T11sqr.pkl','rb')) #Use this if you want the amplitude square as input
    inputtraining = pickle.load(open(os.path.join(out,'T11.pkl'),'rb')) #Use this if you want the real and imaginary part of amplitude as input
    outputtraining = pickle.load(open(os.path.join(out,'labelout.pkl'),'rb'))
    #we may need to convert to float32
    inputtraining = np.float32(np.asarray(inputtraining))
    print('size of dataset:', len(inputtraining))
    print('number of nodes in input layer:',len(inputtraining[0]))
    #shuffle the imported data
    if Nshuffle != 0:
        for ndx in range(Nshuffle):
            inputtraining, outputtraining = shuffle(inputtraining, outputtraining)
        #create input-output tuples that can be read by chainer
        train = chainer.datasets.TupleDataset(inputtraining, outputtraining)
        #split training set with testing set
        pickle.dump (train, open(os.path.join(out,'chainer_train.pkl'),'wb'), protocol=4)
    elif Nshuffle == 0:
        #create input-output tuples that can be read by chainer
        test = chainer.datasets.TupleDataset(inputtraining, outputtraining)
        #split training set with testing set
        pickle.dump (test, open(os.path.join(out,'chainer_test.pkl'),'wb'), protocol=4)
    return